In [3]:
# Some dependencies, you may need to install them, if you do then uncomment the following lines
using Pkg
Pkg.add("Plots")
# Pkg.add("Symbolics")
# Pkg.add("DataFrames")
Pkg.add("CSV");
using Symbolics
using DataFrames
using Plots

# write DataFrame out to CSV file
#CSV.write("dataframe.csv", df)

    Updating registry at `~/.julia/registries/General.toml`


   Resolving package versions...


  No Changes to `~/.julia/environments/v1.9/Project.toml`
  No Changes to `~/.julia/environments/v1.9/Manifest.toml`


   Resolving package versions...


   Installed WorkerUtilities ─ v1.6.1
   Installed WeakRefStrings ── v1.4.2


...

  ✓ WorkerUtilities


  ✓ WeakRefStrings


  ✓ FilePathsBase


  ✓ CSV


  4 dependencies successfully precompiled in 39 seconds. 240 already precompiled. 1 skipped during auto due to previous errors.


In [ ]:
function error_metric(xaprox::Float64, xtarget::Float64, is_absolute::Bool)
    
    """
        This function calculates the error between the aproximation and the real value
        

        Parameters
        ----------
        xaprox: Float64, The aproximation of the of the function.
        xtarget: Float64, The real value we want.
        is_absolute: Bool, If true then the error is absolute, if false then the error is relative.

        Returns
        -------
        Float64, The error between the aproximation and the real value of the root of the function.
    """
    
    if is_absolute
        return abs(xtarget - xaprox) 
    else
        return abs(xtarget - xaprox)/xtarget
    end
    
end

In [ ]:
function find_segment_root(f::Function, x0::Float64, x1::Float64)::Float64
    """
        Finds the root of the segments that join (x0, f(x0)) and (x1, f(x1))
    """
    return x1 - f(x1) * (x1 - x0) / (f(x1) - f(x0))
end


function secant_method(f::Function,
                       x0::Float64,
                       x1::Float64,
                       tol::Float64,
                       maxiter::Int64,)
    """
        Finds the root of a function using the secant method.
        returns: the final [x0, x1] interval where the root is located.
    """
    
    if !check_root_in_the_middle(f, x0, x1)
        return x0, maxiter
    end

   for i in 1:maxiter
        x2 = find_segment_root(f, x0, x1)
        x0 = x1
        x1 = x2

        if abs(f(x1) - f(x0)) < tol
            return x1, i
        end
    end

    return x1, maxiter
end

In [2]:

function build_continuity_coefficients(X::Matrix{Float64}, points::Vector{Vector{Float64}})

    """
        Given a matrix full of zeros, and of size (4n, 4n), where n is the number of polynomials to be found
        for the cubic spline, this function fills the upper part of the matrix corresponding to the equations 
        that say that the polynomials must be continuous at the intersections of the intervals.

        Arguments:
        ----------------
            X: Matrix of zeros of size (4n, 4n)
            points: Vector of points [x,y] to be interpolated
        
        Returns:
        ----------------
            X: Matrix of zeros of size (4n, 4n) with the continuity coefficients filled.
    """

    # Check that the matrix is square
    size(X, 1) == size(X, 2) || error("The matrix must be square")

    row = 1
    n_intervals = length(points) - 1

    # Fill the upper part of the matrix
    for i in 1:n_intervals
        
        # Columns to be filled
        cols = (4 * i - 3):(4 * i)
        
        # Points to get the coefficients to fill the columns with
        start_point = points[i]
        end_point = points[i + 1]

        X[row, cols] = [start_point[1]^3, start_point[1]^2, start_point[1], 1]
        X[row+1, cols] = [end_point[1]^3, end_point[1]^2, end_point[1], 1]
        row += 2
    end
    

    return X
end

function build_first_derivative_continuity_coefficients(X::Matrix{Float64}, points::Vector{Vector{Float64}})

    """
        Given a matrix full of zeros, and of size (4n, 4n), where n is the number of polynomials to be found
        for the cubic spline, this function fills the middle part of the matrix corresponding to the equations 
        that say that the first derivative of the polynomials must be continuous at the intersections of the intervals, 
        this is that the first derivative of the polynomial at the end of an interval must be equal to the first derivative
        of the polynomial at the beginning of the next interval.

        Arguments:
        ----------------
            X: Matrix of zeros of size (4n, 4n)
            points: Vector of points [x,y] to be interpolated
        
        Returns:
        ----------------
            X: Matrix of zeros of size (4n, 4n) with the first derivative continuity coefficients filled.
    """

    # Check that the matrix is square
    size(X, 1) == size(X, 2) || error("The matrix must be square")

    n_intervals = length(points) - 1
    row = 2*(n_intervals)+ 1  # 2(n -1) + 1 

    # Fill the first mid part part of the matrix
    for i in 1:n_intervals-1
        
        # Columns to be filled
        col1 = (4 * i - 3):(4 * i)
        col2 = (4 * i + 1):(4 * i + 4)

        # Points to get the coefficients to fill the columns with
        start_point = points[i+1]

        X[row, col1] = [-3*start_point[1]^2, -2*start_point[1], -1, 0]
        X[row, col2] = [ 3*start_point[1]^2,    2*start_point[1],    1, 0]

        row += 1
    end
    
    return X
end

function build_second_derivative_continuity_coefficients(X::Matrix{Float64}, points::Vector{Vector{Float64}})

    """
        Given a matrix full of zeros, and of size (4n, 4n), where n is the number of polynomials to be found
        for the cubic spline, this function fills the second mid part of the matrix corresponding to the equations 
        that say that the second derivative of the polynomials must be continuous at the intersections of the intervals, 
        this is that the second derivative of the polynomial at the end of an interval must be equal to the second derivative
        of the polynomial at the beginning of the next interval.

        Arguments:
        ----------------
            X: Matrix of zeros of size (4n, 4n)
            points: Vector of points [x,y] to be interpolated
        
        Returns:
        ----------------
            X: Matrix of zeros of size (4n, 4n) with the second derivative continuity coefficients filled.
    """

    # Check that the matrix is square
    size(X, 1) == size(X, 2) || error("The matrix must be square")

    n_intervals = length(points) - 1
    row = 3*(n_intervals) # 3(n-1) = 3(n) -4 +1

    # Fill the second mid part part of the matrix
    for i in 1:n_intervals-1
        
        # Columns to be filled
        col1 = (4 * i - 3):(4 * i)
        col2 = (4 * i + 1):(4 * i + 4)

        # Points to get the coefficients to fill the columns with
        start_point = points[i+1]

        X[row, col1] = [-6*start_point[1], -2, 0, 0]
        X[row, col2] = [ 6*start_point[1],  2, 0, 0]

        row += 1
    end
    
    return X

end

function build_second_derivative_natural_coefficients(X::Matrix{Float64}, points::Vector{Vector{Float64}})

    """
        Given a matrix full of zeros, and of size (4n, 4n), where n is the number of polynomials to be found
        for the cubic spline, this function fills the second mid part of the matrix corresponding to the equations 
        that say that the second derivative of the polynomials must be equal to zero at the beginning and end of the 
        interval.

        Arguments:
        ----------------
            X: Matrix of zeros of size (4n, 4n)
            points: Vector of points [x,y] to be interpolated
        
        Returns:
        ----------------
            X: Matrix of zeros of size (4n, 4n) with the second derivative natural coefficients filled.
    """

    # Check that the matrix is square
    size(X, 1) == size(X, 2) || error("The matrix must be square")

    n_intervals = length(points) - 1
    row = 2*(2*(n_intervals+1)-3) + 1  # 2(2(n)-3) + 1

    # Columns to be filled
    col1 = 1:4
    col2 = (n_intervals*4 - 3):(n_intervals*4)

    # Points to get the coefficients to fill the columns with
    start_point = points[1]
    end_point = points[end]

    # Fill the second mid part part of the matrix
    X[row, col1] = [6*start_point[1], 2, 0, 0]
    X[row+1,col2] = [6*end_point[1], 2, 0, 0]

    return X
end


function build_cubic_spline_matrix(points::Vector{Vector{Float64}})

    """
        Given a vector of points [x,y] to be interpolated, this function builds the matrix of coefficients
        for the cubic spline method.

        Arguments:
        ----------------
            points: Vector of points [x,y] to be interpolated
        
        Returns:
        ----------------
            X: Matrix of coefficients for the cubic spline method.
    """

    n_intervals = length(points) - 1

    #matrix of zeros of size (4n, 4n)
    X = zeros(4 * n_intervals, 4 * n_intervals)

    #fill the upper part of the matrix and print the complete matrix
    X = build_continuity_coefficients(X, points)
    X = build_first_derivative_continuity_coefficients(X, points)
    X = build_second_derivative_continuity_coefficients(X, points)
    X = build_second_derivative_natural_coefficients(X, points)

    return X
end

function build_output_vector(points::Vector{Vector{Float64}})

    """
        Given a vector of points [x,y] to be interpolated, this function builds the output vector
        for the cubic spline method.

        Arguments:
        ----------------
            points: Vector of points [x,y] to be interpolated.
        
        Returns:
        ----------------
            Y: Vector of coefficients for the cubic spline method.
    """

    n_intervals = length(points) - 1

    #vector of zeros of size (4n, 1)
    Y = zeros(4 * n_intervals, 1)


    #fill the upper part of the matrix and print the complete matrix
    Y[1] = points[1][2]
    Y[2*(n_intervals-1)+2] = points[end][2]

    row = 2
    for i in 2:n_intervals
        Y[row] = points[i][2]
        Y[row+1] = points[i][2]
        row += 2
    end

    return Y

end


function build_and_solve_coefficients(points::Vector{Vector{Float64}})::Matrix{Float64}

    """
        Given a vector of points [x,y] to be interpolated, this function builds the matrix of coefficients
        and the output vector for the cubic spline method, and then solves the system of equations.

        Arguments:
        ----------------
            points: Vector of points [x,y] to be interpolated.
        
        Returns:
        ----------------
            coefficients: Vector of coefficients for the cubic spline method.
    """


    X = build_cubic_spline_matrix(points)
    Y = build_output_vector(points)
    return X\Y
end

function build_cubic_splines(points::Vector{Vector{Float64}}, x::Num)::Vector{Tuple{Function, Float64, Float64}}

    """
        Given a vector of points [x,y] to be interpolated, this function builds the cubic splines
        for the cubic spline method.

        Arguments:
        ----------------
            points: Vector of points [x,y] to be interpolated.
            x: Symbolics variable to build the polynomials
        
        Returns:
        ----------------
            polynomials: Vector of polynomials for the cubic spline method.
    """

    #sort points by x value 
    points = sort(points, by = x -> x[1])
    
    #build and solve the coefficients
    coefficients = build_and_solve_coefficients(points)
    n_intervals = length(points) - 1
    polynomials = []

    #build the polynomials
    for i in 1:n_intervals
        start_point = points[i]
        end_point = points[i+1]
        polynomial = coefficients[4*i-3]*x^3 + coefficients[4*i-2]*x^2 + coefficients[4*i-1]*x + coefficients[4*i]
        push!(polynomials, (Symbolics.build_function(polynomial, x, expression = false) , start_point[1],end_point[1]))
    end
    return polynomials
end


function interpolate(polynomals::Vector{Tuple{Function, Float64, Float64}}, x::Float64)::Float64
    """
        Given a vector of polynomials, this function interpolates the value of x using the polynomials.

        Arguments:
        ----------------
            polynomals: Vector of polynomials for the cubic spline method.
            x: value to interpolate
        
        Returns:
        ----------------
            y: interpolated value.
    """
    for polynomial in polynomals
        if x >= polynomial[2] && x <= polynomial[3]
            return polynomial[1](x)
        end
    end
    return nothing
end

interpolate (generic function with 1 method)